<a href="https://colab.research.google.com/github/angelatyk/tinytutor/blob/dev/notebooks/00_master_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies

In [1]:
!pip install -q google-adk google-generativeai python-dotenv
!pip install -q google-cloud-texttospeech pydub
!pip install -q gradio

print("✅ All libraries installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.2/192.2 kB 3.8 MB/s eta 0:00:00
✅ All libraries installed.


## Libraries Import

In [2]:
import os
import json
import asyncio
from pathlib import Path
from typing import List, Tuple

import google.generativeai as genai
from google.colab import userdata

from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

from google.cloud import texttospeech
from pydub import AudioSegment

import gradio as gr

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


## Configure API Keys

In [3]:
# Gemini Key
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
genai.configure(api_key=GOOGLE_API_KEY)
print("✅ Gemini API configured.")

# Google TTS Service Account JSON
SERVICE_ACCOUNT_JSON = userdata.get("GCP_VI_SERVICE_ACCOUNT_JSON")
if not SERVICE_ACCOUNT_JSON:
    raise RuntimeError("Upload GCP_VI_SERVICE_ACCOUNT_JSON to Colab Secrets!")

with open("tinytutor-tss-agent.json", "w") as f:
    f.write(SERVICE_ACCOUNT_JSON)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "tinytutor-tss-agent.json"

tts_client = texttospeech.TextToSpeechClient()
print("✅ Google TTS configured.")

✅ Gemini API configured.
✅ Google TTS configured.


## Configure Retry Options

In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

## PedagogyAgent

In [5]:
pedagogy_agent = Agent(
    name="PedagogyAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Explains topics in simple ELI5 style.",
    instruction="Explain the topic like I'm 5. Use google_search if needed.",
    tools=[google_search],
)

runner = InMemoryRunner(agent=pedagogy_agent)

In [6]:
async def run_pedagogy_async(topic: str) -> str:
    response = await runner.run_debug(topic)

    return response[0].content.parts[0].text

## AudioScriptWriterAgent

In [7]:
SCRIPTWRITER_SYSTEM_PROMPT = """
You are AudioNarratorAgent. Produce a single-voice children’s story (200–450 words).
No labels, no titles, no markdown. Add exactly 2 learning questions inside the story.
"""

def run_scriptwriter(explanation: str) -> str:
    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash",
        system_instruction=SCRIPTWRITER_SYSTEM_PROMPT
    )

    response = model.generate_content(
        f"Write a children's story based on this:\n{explanation}",
        generation_config=genai.GenerationConfig(
            temperature=0.9,
            max_output_tokens=4096
        )
    )

    # Safest extraction
    try:
        return response.text
    except Exception:
        pass

    # Fallback
    try:
        return response.candidates[0].content.parts[0].text
    except Exception:
        pass

    return "⚠️ ScriptWriter failed."

## AudioGeneratorAgent

In [8]:
def chunk_text(text, max_chars=4500):
    text = text.strip()
    if len(text) <= max_chars:
        return [text]
    chunks = []
    while len(text) > max_chars:
        cut = text.rfind(". ", 0, max_chars)
        if cut == -1:
            cut = max_chars
        chunks.append(text[:cut+1])
        text = text[cut+1:].strip()
    chunks.append(text)
    return chunks


def tts_segment(text):
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Neural2-C"
    )
    audio_cfg = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3,
        speaking_rate=1.02,
        pitch=-2.0
    )
    response = tts_client.synthesize_speech(
        input=synthesis_input,
        voice=voice,
        audio_config=audio_cfg
    )
    return response.audio_content


def audio_writer(script_text: str, out="story.mp3"):
    chunks = chunk_text(script_text)
    audio = AudioSegment.silent(200)
    for i, chunk in enumerate(chunks, 1):
        path = f"seg_{i}.mp3"
        with open(path, "wb") as f:
            f.write(tts_segment(chunk))
        audio += AudioSegment.from_mp3(path)
        audio += AudioSegment.silent(150)
    audio.export(out, format="mp3")
    return out


## Gradio

In [9]:
async def full_pipeline(topic: str):
    eli5 = await run_pedagogy_async(topic)
    script = run_scriptwriter(eli5)
    audio_path = audio_writer(script, "story.mp3")
    return eli5, script, audio_path

In [ ]:
app = gr.Interface(
    fn=full_pipeline,
    inputs=gr.Textbox(label="Your Topic"),
    outputs=[
        gr.Textbox(label="ELI5 Explanation", lines=8),
        gr.Textbox(label="Generated Story Script", lines=20),
        gr.Audio(label="Generated Audio")
    ],
    title="🎧 TinyTutor — Full Pipeline",
    css=".gradio-container { min-height: 1200px !important; }"
)

app.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2f68a35cced9651fb7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



 ### Created new session: debug_session_id

User > What is Agentic AI?
PedagogyAgent > Imagine you have a super-smart toy robot. You tell it to tidy your room.

Instead of just waiting for you to tell it *exactly* what to do (like "pick up the blue car," then "put it in the toy box"), this smart robot can *figure things out on its own*!

It sees the mess, remembers what a tidy room looks like, and then makes a plan. It might decide to pick up the blocks first, then the dolls, and then put them all away. It can even ask you questions if it's not sure about something, like "Should I put this drawing in the bin?"

**Agentic AI** is like that super-smart robot. It's a type of computer brain that can:

*   **See and understand** what's happening around it (like seeing the messy room).
*   **Think for itself** and make a plan to do something (like deciding to pick up blocks first).
*   **Take actions** to reach a goal (like actually picking up the blocks and putting them away).
*   **Learn 